In [ ]:
%load_ext cudf.pandas
import pandas as pd
from llama_cpp import Llama
import re
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
from scipy.stats import spearmanr, kendalltau

In [ ]:
cerbero = Llama(
    model_path='ggml-model-Q8_0.gguf',
    n_ctx=4086,
    n_gpu_layers=21
)

bugiardini = pd.read_table('ground_truth.tsv')

In [ ]:
answers_prompt1 = []
for i, farmaco in enumerate(bugiardini['farmaco']):
    print('Sample n.', i)
    #prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
#[|Umano|] Cos'è il farmaco {farmaco} e a cosa serve?
#[|Assistente|]'''
    prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
[|Umano|] A cosa serve {farmaco}?
[|Assistente|]'''
    output = cerbero(prompt, max_tokens=None, stop='[|Umano|]')
    answers_prompt1.append(output['choices'][0]['text'].strip(' ').strip('\n'))

answers1 = pd.DataFrame({'answers_prompt_1':answers_prompt1
                        })
#answers1.to_csv('/home/luca/Scrivania/answers1_ZSP1.tsv', sep='\t')
answers1.to_csv('/home/luca/Scrivania/answers1_ZSP2.tsv', sep='\t')

In [ ]:
answers_prompt2 = []
for i, farmaco in enumerate(bugiardini['farmaco']):
    print('Sample n.', i)
    #prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
#[|Umano|] Ci sono particolari avvertenze, precauzioni o potenziali interazioni con altri farmaci per il farmaco {farmaco}?
#[|Assistente|]'''
    prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
[|Umano|] Ci sono controindicazioni per {farmaco}?
[|Assistente|]'''
    output = cerbero(prompt, max_tokens=None, stop='[|Umano|]')
    answers_prompt2.append(output['choices'][0]['text'].strip(' ').strip('\n'))

answers2 = pd.DataFrame({'answers_prompt_2':answers_prompt2
                        })
#answers2.to_csv('/home/luca/Scrivania/answers2_ZSP1.tsv', sep='\t')
answers2.to_csv('/home/luca/Scrivania/answers2_ZSP2.tsv', sep='\t')

In [ ]:
answers_prompt3 = []
for i, farmaco in enumerate(bugiardini['farmaco']):
    print('Sample n.', i)
#    prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
#[|Umano|] Quando, in quali dosi e in che modo devo prendere il farmaco {farmaco}?
#[|Assistente|]'''
    prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
[|Umano|] Come devo assumere {farmaco}?
[|Assistente|]'''
    output = cerbero(prompt, max_tokens=None, stop='[|Umano|]')
    answers_prompt3.append(output['choices'][0]['text'].strip(' ').strip('\n'))

answers3 = pd.DataFrame({'answers_prompt_3':answers_prompt3
                        })
#answers3.to_csv('/home/luca/Scrivania/answers3_ZSP1.tsv', sep='\t')
answers3.to_csv('/home/luca/Scrivania/answers3_ZSP2.tsv', sep='\t')

In [ ]:
answers_prompt4 = []
for i, farmaco in enumerate(bugiardini['farmaco']):
    print('Sample n.', i)
    #prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
#[|Umano|] Quali sono i possibili effetti indesiderati del farmaco {farmaco}?
#[|Assistente|]'''
    prompt = f'''Questa è una conversazione tra un umano ed un assistente AI. L'assistente AI risponde con parole semplici alle domande dei pazienti sui farmaci.
[|Umano|] Ci sono effetti collaterali per {farmaco}?
[|Assistente|]'''
    output = cerbero(prompt, max_tokens=None, stop='[|Umano|]')
    answers_prompt4.append(output['choices'][0]['text'].strip(' ').strip('\n'))

answers4 = pd.DataFrame({'answers_prompt_4':answers_prompt4
                        })
#answers4.to_csv('/home/luca/Scrivania/answers4_ZSP1.tsv', sep='\t')
answers4.to_csv('/home/luca/Scrivania/answers4_ZSP2.tsv', sep='\t')

In [ ]:
'''answers_ZSP1 = pd.concat([
    answers1,
    answers2,
    answers3,
    answers4
],axis=1)'''

answers_ZSP2 = pd.concat([
    answers1,
    answers2,
    answers3,
    answers4
],axis=1)

In [ ]:
def clean_answers(txt):
    if '[|Umano|]' in txt:
        txt = txt.split('[|Umano|]')[0].strip()
    elif '[| Umano |]' in txt:
        txt = txt.split('[| Umano |]')[0].strip()
    elif '[| Umano|]' in txt:
        txt = txt.split('[| Umano|]')[0].strip()
    elif '[|Umano |]' in txt:
        txt = txt.split('[|Umano |]')[0].strip()

    elif '[|Human|]' in txt:
        txt = txt.split('[|Human|]')[0].strip()
    elif '[| Human |]' in txt:
        txt = txt.split('[| Human |]')[0].strip()
    elif '[| Human|]' in txt:
        txt = txt.split('[| Human|]')[0].strip()
    elif '[|Human |]' in txt:
        txt = txt.split('[|Human |]')[0].strip()

    elif '[|Humano|]' in txt:
        txt = txt.split('[|Humano|]')[0].strip()
    elif '[| Humano |]' in txt:
        txt = txt.split('[| Humano |]')[0].strip()
    elif '[| Humano|]' in txt:
        txt = txt.split('[| Humano|]')[0].strip()
    elif '[|Humano |]' in txt:
        txt = txt.split('[|Humano |]')[0].strip()

    if '[|Assistente|]' in txt:
        txt = txt.split('[|Assistente|]')[0].strip()
    elif '[| Assistente |]' in txt:
        txt = txt.split('[| Assistente |]')[0].strip()
    elif '[| Assistente|]' in txt:
        txt = txt.split('[| Assistente|]')[0].strip()
    elif '[|Assistente |]' in txt:
        txt = txt.split('[|Assistente |]')[0].strip()
    
    return txt

#answers_ZSP1['answers_prompt_1'] = answers_ZSP1['answers_prompt_1'].progress_apply(clean_answers)
#answers_ZSP1['answers_prompt_2'] = answers_ZSP1['answers_prompt_2'].progress_apply(clean_answers)
#answers_ZSP1['answers_prompt_3'] = answers_ZSP1['answers_prompt_3'].progress_apply(clean_answers)
#answers_ZSP1['answers_prompt_4'] = answers_ZSP1['answers_prompt_4'].progress_apply(clean_answers)

answers_ZSP2['answers_prompt_1'] = answers_ZSP2['answers_prompt_1'].progress_apply(clean_answers)
answers_ZSP2['answers_prompt_2'] = answers_ZSP2['answers_prompt_2'].progress_apply(clean_answers)
answers_ZSP2['answers_prompt_3'] = answers_ZSP2['answers_prompt_3'].progress_apply(clean_answers)
answers_ZSP2['answers_prompt_4'] = answers_ZSP2['answers_prompt_4'].progress_apply(clean_answers)

In [ ]:
###############################
### EVALUATION - SIMILARITY ###
###############################

def get_text_embeddings(text):
    sentences = re.findall('[^!?。.？！]+[!?。.？！]?', text)
    sentence_embeddings = sbert.encode(sentences)
    text_embeddings = np.mean(sentence_embeddings, axis=0)

    return text_embeddings

sbert = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device='cuda')

### PROMPT 1

df = pd.DataFrame()
df['introduzione_embeddings'] = bugiardini['introduzione'].progress_apply(get_text_embeddings)
#df['answers_prompt_1_embeddings'] = answers_ZSP1['answers_prompt_1'].progress_apply(get_text_embeddings)
df['answers_prompt_1_embeddings'] = answers_ZSP2['answers_prompt_1'].progress_apply(get_text_embeddings)

cosine_similarities1 = [util.cos_sim(np.array([emb1]), np.array([emb2]))[0][0] for emb1, emb2 in zip(df['introduzione_embeddings'], df['answers_prompt_1_embeddings'])]

### PROMPT 2

df['avv_prec_inter_embeddings'] = bugiardini['avv_prec_inter'].progress_apply(get_text_embeddings)
#df['answers_prompt_2_embeddings'] = answers_ZSP1['answers_prompt_2'].progress_apply(get_text_embeddings)
df['answers_prompt_2_embeddings'] = answers_ZSP2['answers_prompt_2'].progress_apply(get_text_embeddings)

cosine_similarities2 = [util.cos_sim(np.array([emb1]), np.array([emb2]))[0][0] for emb1, emb2 in zip(df['avv_prec_inter_embeddings'], df['answers_prompt_2_embeddings'])]

### PROMPT 3

df['dosaggio_embeddings'] = bugiardini['dosaggio'].progress_apply(get_text_embeddings)
#df['answers_prompt_3_embeddings'] = answers_ZSP1['answers_prompt_3'].progress_apply(get_text_embeddings)
df['answers_prompt_3_embeddings'] = answers_ZSP2['answers_prompt_3'].progress_apply(get_text_embeddings)

cosine_similarities3 = [util.cos_sim(np.array([emb1]), np.array([emb2]))[0][0] for emb1, emb2 in zip(df['dosaggio_embeddings'], df['answers_prompt_3_embeddings'])]

### PROMPT 4
df = pd.DataFrame()
df['effetti_indesiderati_embeddings'] = bugiardini['effetti_indesiderati'].progress_apply(get_text_embeddings)
#df['answers_prompt_4_embeddings'] = answers_ZSP1['answers_prompt_4'].progress_apply(get_text_embeddings)
df['answers_prompt_4_embeddings'] = answers_ZSP2['answers_prompt_4'].progress_apply(get_text_embeddings)

cosine_similarities4 = [util.cos_sim(np.array([emb1]), np.array([emb2]))[0][0] for emb1, emb2 in zip(df['effetti_indesiderati_embeddings'], df['answers_prompt_4_embeddings'])]

cos_simi = pd.DataFrame({
    'sim1':cosine_similarities1,
    'sim2':cosine_similarities2,
    'sim3':cosine_similarities3,
    'sim4':cosine_similarities4
    })

#cos_simi.to_csv('cosine_similarities_ZSP1.tsv', sep='\t')
cos_simi.to_csv('cosine_similarities_ZSP2.tsv', sep='\t')

In [ ]:
#answers = pd.concat([answers_ZSP1, cos_simi], axis=1)
#answers.to_csv('answers_ZSP1_clean_\&_similarity.tsv')

answers = pd.concat([answers_ZSP2, cos_simi], axis=1)
answers.to_csv('answers_ZSP2_clean_\&_similarity.tsv')

In [ ]:
###################################
### EVALUATION - NER OVERLAP F1 ###
###################################

tokenizer = AutoTokenizer.from_pretrained("DeepMount00/Italian_NER_XXL")
model = AutoModelForTokenClassification.from_pretrained("DeepMount00/Italian_NER_XXL", ignore_mismatched_sizes=True)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

tag_list = ['B-ETA', 'I-ETA', 'B-MALATTIA', 'I-MALATTIA', 'B-MEDICINA', 'I-MEDICINA',
            'B-STRENGTH', 'I-STRENGTH', 'B-FREQUENZA', 'I-FREQUENZA', 'B-DURATION',
            'I-DURATION', 'B-DOSAGGIO', 'I-DOSAGGIO', 'B-FORM', 'I-FORM']

ground_truth = pd.read_table('ground_truth.tsv')
#answers = pd.read_table('answers_ZSP1_clean_&_similarity.tsv')
answers = pd.read_table('answers_ZSP2_clean_&_similarity.tsv')

In [ ]:
def calculate_metrics_single_pair(gold, answer):
    ent_goldstandard = {d['word'] for d in nlp(gold.lower()) if d['word'] != '.' and d['entity'] in tag_list}
    ent_answer = {d['word'] for d in nlp(answer.lower()) if d['word'] != '.' and d['entity'] in tag_list}
    
    common_entities = ent_goldstandard.intersection(ent_answer)

    precision = len(common_entities) / len(ent_answer) if len(ent_answer) != 0 else 0
    recall = len(common_entities) / len(ent_goldstandard) if len(ent_goldstandard) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    
    return precision, recall, f1_score, ent_goldstandard, ent_answer

def calculate_metrics_parallel(gold_standard, answers):
    metrics_list = []
    with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        for metrics in tqdm(executor.map(calculate_metrics_single_pair, gold_standard, answers), total=len(gold_standard)):
            metrics_list.append(metrics)
    return metrics_list

metrics = calculate_metrics_parallel(ground_truth['introduzione'], answers['answers_prompt_1'])
answers[['Precision_1', 'Recall_1', 'F1_1', 'ent_gold_1', 'ent_output_1']] = pd.DataFrame(metrics, columns=['Precision', 'Recall', 'F1', 'ent_gold', 'ent_output'])

metrics = calculate_metrics_parallel(ground_truth['avv_prec_inter'], answers['answers_prompt_2'])
answers[['Precision_2', 'Recall_2', 'F1_2', 'ent_gold_2', 'ent_output_2']] = pd.DataFrame(metrics, columns=['Precision', 'Recall', 'F1', 'ent_gold', 'ent_output'])

metrics = calculate_metrics_parallel(ground_truth['dosaggio'], answers['answers_prompt_3'])
answers[['Precision_3', 'Recall_3', 'F1_3', 'ent_gold_3', 'ent_output_3']] = pd.DataFrame(metrics, columns=['Precision', 'Recall', 'F1', 'ent_gold', 'ent_output'])

metrics = calculate_metrics_parallel(ground_truth['effetti_indesiderati'], answers['answers_prompt_4'])
answers[['Precision_4', 'Recall_4', 'F1_4', 'ent_gold_4', 'ent_output_4']] = pd.DataFrame(metrics, columns=['Precision', 'Recall', 'F1', 'ent_gold', 'ent_output'])

#answers.to_csv('ZSP1_NER_f1_precision_recall_ents.tsv', sep='\t')
answers.to_csv('ZSP2_NER_f1_precision_recall_ents.tsv', sep='\t')

In [ ]:
for i,j in zip(['sim1', 'sim2', 'sim3', 'sim4'], ['F1_1', 'F1_2', 'F1_3', 'F1_4']):
    #sim = pd.read_table('answers_ZSP1_clean_&_similarity.tsv')[i]
    sim = pd.read_table('answers_ZSP2_clean_&_similarity.tsv')[i]
    f1 = pd.read_table('ZSP2_NER_f1_precision_recall_ents.tsv')[j]

    correlation_spearman, p_value_spearman = spearmanr(sim, f1)
    correlation_kendalltau, p_value_kendalltau = kendalltau(sim, f1)

    print(f"Spearman's rank correlation coefficient: {correlation_spearman}")
    print(f"P-value: {p_value_spearman}")

    print(f"Kendall's Tau: {correlation_kendalltau}")
    print(f"P-value: {p_value_kendalltau}")